# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

In [217]:
%pip install numpy pandas



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [218]:
import numpy as np
import pandas as pd


## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.

In [219]:
# Read data
df = pd.read_csv("strawberry-prices.tsv", sep="\t")
df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").reset_index(drop=True)

df.head()


,month,price
0,2020-01-01,4.0490
1,2020-02-01,3.6250
2,2020-03-01,3.3770
3,2020-04-01,3.2328
4,2020-05-01,3.1260


In [220]:
train = df[(df["month"] >= "2020-01-01") & (df["month"] <= "2023-12-01")].copy()
train["m"] = train["month"].dt.month

month_avg = train.groupby("m")["price"].mean()

months_2024 = pd.date_range("2024-01-01", "2024-12-01", freq="MS")
backtest = pd.DataFrame({
    "month": months_2024.strftime("%Y-%m-01"),
    "price": [month_avg.loc[m] for m in months_2024.month]
})

backtest.to_csv("strawberry-backtest.tsv", sep="\t", index=False)
backtest.head()


,month,price
0,2024-01-01,4.36275
1,2024-02-01,4.09125
2,2024-03-01,3.68875
3,2024-04-01,3.78720
4,2024-05-01,3.52625


Please use the same format for the month column as in the training data, i.e. YYYY-MM-01.
The autograder may not be able to parse other formats.

Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth?

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [221]:
truth = pd.read_csv("strawberry-prices.tsv", sep="\t")
pred = pd.read_csv("strawberry-backtest.tsv", sep="\t")

truth["month"] = pd.to_datetime(truth["month"])
pred["month"] = pd.to_datetime(pred["month"])

truth_2024 = truth[(truth["month"] >= "2024-01-01") & (truth["month"] <= "2024-12-01")].copy()
truth_2024 = truth_2024.sort_values("month").reset_index(drop=True)

pred = pred.sort_values("month").reset_index(drop=True)

residuals = truth_2024["price"] - pred["price"]

mean_resid = residuals.mean()
std_resid = residuals.std(ddof=1)

accuracy = pd.DataFrame({"mean": [mean_resid], "std": [std_resid]})
accuracy.to_csv("backtest-accuracy.tsv", sep="\t", index=False)

accuracy


,mean,std
0,-0.004856,0.274566


In [222]:
timings = pd.read_csv("timings.tsv", sep="\t")
accuracy = pd.read_csv("backtest-accuracy.tsv", sep="\t")

best_row = timings.loc[timings["expected_profit"].idxmax()]
best_profit = best_row["expected_profit"]
best_pints = best_row["pints_purchased"]

std_resid = accuracy.loc[0, "std"]
one_std_profit = best_pints * std_resid

check = pd.DataFrame({"best_profit": [best_profit], "one_std_profit": [one_std_profit]})
check.to_csv("check.tsv", sep="\t", index=False)

check


,best_profit,one_std_profit
0,114385.703292,70793.566481


Hint: If the mean residual in your backtest is not close to zero, then your model is likely missing a systematic change and you should go back to improve it.

Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.


In [223]:
df = pd.read_csv("strawberry-prices.tsv", sep="\t")
df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").reset_index(drop=True)


In [224]:
train = df[(df["month"] >= "2020-01-01") & (df["month"] <= "2024-12-01")].copy()
train["m"] = train["month"].dt.month

train.shape

(60, 3)

In [225]:
month_avg = train.groupby("m")["price"].mean()
month_avg


m
1     4.50120
2     4.12580
3     3.69940
4     3.74496
5     3.46840
6     3.21540
7     3.17840
8     3.46120
9     3.61380
10    3.88440
11    4.40240
12    4.80226
Name: price, dtype: float64

In [226]:
months_2025 = pd.date_range("2025-01-01", "2025-12-01", freq="MS")

forecast = pd.DataFrame({
    "month": months_2025.strftime("%Y-%m-01"),
    "price": [month_avg.loc[m] for m in months_2025.month]
})

forecast


,month,price
0,2025-01-01,4.50120
1,2025-02-01,4.12580
2,2025-03-01,3.69940
3,2025-04-01,3.74496
4,2025-05-01,3.46840
5,2025-06-01,3.21540
6,2025-07-01,3.17840
7,2025-08-01,3.46120
8,2025-09-01,3.61380
9,2025-10-01,3.88440


In [227]:
forecast.to_csv("strawberry-forecast.tsv", sep="\t", index=False)


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [228]:
forecast = pd.read_csv("strawberry-forecast.tsv", sep="\t")
forecast["month"] = pd.to_datetime(forecast["month"])
forecast = forecast.sort_values("month").reset_index(drop=True)

forecast

,month,price
0,2025-01-01,4.50120
1,2025-02-01,4.12580
2,2025-03-01,3.69940
3,2025-04-01,3.74496
4,2025-05-01,3.46840
5,2025-06-01,3.21540
6,2025-07-01,3.17840
7,2025-08-01,3.46120
8,2025-09-01,3.61380
9,2025-10-01,3.88440


In [229]:
CAPITAL = 1_000_000
FREEZE_COST = 0.20
STORAGE_COST = 0.10
DISCOUNT = 0.10


In [230]:
rows = []

for i, buy_row in forecast.iterrows():
    for j, sell_row in forecast.iterrows():
        
        if j <= i:
            continue
        
        buy_month = buy_row["month"].strftime("%Y-%m-01")
        sell_month = sell_row["month"].strftime("%Y-%m-01")
        
        buy_price = buy_row["price"]
        sell_price = sell_row["price"]
        
        months_stored = j - i
        
        # cost per pint
        cost_per_pint = (
            buy_price
            + FREEZE_COST
            + STORAGE_COST * months_stored
        )
        
        # revenue per pint (10% discount)
        revenue_per_pint = sell_price * (1 - DISCOUNT)
        
        profit_per_pint = revenue_per_pint - cost_per_pint
        
        # how many pints can we afford?
        pints_purchased = int(CAPITAL // cost_per_pint)
        
        expected_profit = pints_purchased * profit_per_pint
        
        rows.append([
            buy_month,
            sell_month,
            pints_purchased,
            expected_profit
        ])


In [231]:
timings = pd.DataFrame(
    rows,
    columns=["buy_month", "sell_month", "pints_purchased", "expected_profit"]
)

timings.head()


,buy_month,sell_month,pints_purchased,expected_profit
0,2025-01-01,2025-02-01,208281,-226605.562380
1,2025-01-01,2025-03-01,204031,-320683.683940
2,2025-01-01,2025-04-01,199952,-326068.924672
3,2025-01-01,2025-05-01,196032,-388072.788480
4,2025-01-01,2025-06-01,192263,-443616.110420


In [232]:
timings.to_csv("timings.tsv", sep="\t", index=False)


In [233]:
timings.sort_values("expected_profit", ascending=False).head(10)


,buy_month,sell_month,pints_purchased,expected_profit
55,2025-07-01,2025-12-01,257838,114385.703292
50,2025-06-01,2025-12-01,249041,76364.437994
59,2025-08-01,2025-12-01,246232,64225.677488
62,2025-09-01,2025-12-01,243084,50618.353656
54,2025-07-01,2025-11-01,264662,48634.289120
49,2025-06-01,2025-11-01,255401,11942.550760
64,2025-10-01,2025-12-01,233404,8783.926136
58,2025-08-01,2025-11-01,252448,242.350080
44,2025-05-01,2025-12-01,228916,-10613.919256
61,2025-09-01,2025-11-01,249140,-12865.589600


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns `best_profit` and `one_std_profit`.
To be clear, `one_std_profit` should be the number of pints bought in your best profit scenario times your backtested standard deviation of the residual.
This represents the standard deviation in revenue when selling if you explicitly assume that you buy according to the best profit scenario and your backtest standard deviation is representative of the future prices.

In [234]:
timings = pd.read_csv("timings.tsv", sep="\t")
accuracy = pd.read_csv("backtest-accuracy.tsv", sep="\t")

In [235]:
best_row = timings.loc[timings["expected_profit"].idxmax()]

best_profit = best_row["expected_profit"]
best_pints = best_row["pints_purchased"]

best_row


buy_month             2025-07-01
sell_month            2025-12-01
pints_purchased           257838
expected_profit    114385.703292
Name: 55, dtype: object

In [236]:
std_resid = accuracy.loc[0, "std"]

one_std_profit = best_pints * std_resid

best_profit, one_std_profit


(np.float64(114385.70329200008), np.float64(70793.56648081228))

In [237]:
check = pd.DataFrame({
    "best_profit": [best_profit],
    "one_std_profit": [one_std_profit]
})

check.to_csv("check.tsv", sep="\t", index=False)
check


,best_profit,one_std_profit
0,114385.703292,70793.566481


Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.